In [1]:
%load_ext autoreload
%autoreload 2

from keras import optimizers
from keras.callbacks import CSVLogger
import numpy as np
import os
import glob
import sys
import librosa

sys.path.insert(0,'../../..')
from sed_endtoend.pcen.model import SMel_PCEN, concatenate_stft
from sed_endtoend.cnn.model import build_custom_cnn
from sed_endtoend.data_generator import DataGenerator, Scaler
from sed_endtoend.optimizer import Adam_Multipliers
from sed_endtoend.callbacks import MetricsCallback
from sed_endtoend.gen_mel_filters import mel_filters

from params import *

os.environ["CUDA_VISIBLE_DEVICES"]="1"

# files parameters
Nfiles = None
resume = False
load_subset = Nfiles

Using TensorFlow backend.


In [2]:
params = {'sequence_time': sequence_time, 'sequence_hop_time':sequence_hop_time,
          'label_list':label_list,'audio_hop':audio_hop, 'audio_win':audio_win,
          'n_fft':n_fft,'sr':sr,'mel_bands':mel_bands,'normalize':normalize_data, 
          'frames':frames,'get_annotations':get_annotations, 'dataset': dataset}

sequence_frames = int(np.ceil(sequence_time*sr/audio_hop))

# Datasets
labels = {}# Labels

train_files = sorted(glob.glob(os.path.join(audio_folder,'train', '*.wav')))
val_files = sorted(glob.glob(os.path.join(audio_folder,'validate', '*.wav')))

if load_subset is not None:
    train_files = train_files[:load_subset]
    val_files = val_files[:load_subset]

train_labels = {}
train_mel = {}
val_labels = {}
val_mel = {}

for n,id in enumerate(train_files):
    labels[id] = os.path.join(label_folder, 'train',os.path.basename(id).replace('.wav','.txt'))
for id in val_files:
    labels[id] = os.path.join(label_folder, 'validate',os.path.basename(id).replace('.wav','.txt'))

# Generators

params['normalize_energy'] = False
params['convert_to_dB'] = False

print('Making training generator')
training_generator = DataGenerator(train_files, labels, **params)

params['sequence_hop_time'] = sequence_time # To calculate F1_1s

print('Making validation generator')
validation_generator = DataGenerator(val_files, labels, **params)

print('Getting validation data')

_,S_val,mel_val,y_val = validation_generator.return_all()

print('Getting training data')

_,S_train,mel_train,y_train = training_generator.return_all()

for j in range(mel_train.shape[0]):
    mel_train[j] = librosa.pcen(mel_train[j],sr=sr, hop_length=audio_hop, 
                                gain=alpha2, bias=delta, power=r, time_constant=time_constant) 

for j in range(mel_val.shape[0]):   
    mel_val[j] = librosa.pcen(mel_val[j],sr=sr, hop_length=audio_hop, 
                                gain=alpha2, bias=delta, power=r, time_constant=time_constant) 

Making training generator
Making validation generator
Getting validation data
0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
Getting training data
0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %


In [4]:
model_mel = SMel_PCEN(mel_bands,sequence_frames,audio_win,audio_hop)    
model_cnn = build_custom_cnn(n_freq_cnn=mel_bands, n_frames_cnn=sequence_frames,large_cnn=large_cnn)

model_cnn.load_weights("../02_train_CNN/weights_best.hdf5")
model_mel.load_weights("../01_train_SMel/weights_best.hdf5")

model = concatenate_stft(sequence_frames,513,model_cnn,model_mel)

#model_cnn.trainable = False

model_mel.summary()
model_cnn.summary()
model.summary()

opt = Adam_Multipliers(lr=learning_rate,multipliers=4*[lr_multiplier]+18*[1])
#opt = Adam(lr=learning_rate)

if resume:
    print('Loading best weights and resuming...')
    weights_best_file = os.path.join(expfolder, 'weights_best.hdf5')
    model.load_weights(weights_best_file)

# Fit model
print('\nFitting model...')

if resume:
    f1s_best = resume_f1_best

metrics_callback = MetricsCallback(S_val, [y_val,mel_val], 0, 0, os.path.join(expfolder, 'weights_best.hdf5'))
#save_fig = SaveFigCallback(train_example,train_y_example,val_example,val_y_example,'./imgs/')
csv_logger = CSVLogger(os.path.join(expfolder,'training.log'))
#alpha = K.variable(0.0)
#beta = K.variable(1.0)
#bt_callback = BT_strategy(alpha,beta, a=8, b=1, W_end=1)

model.compile(loss=['binary_crossentropy','mean_squared_error'],loss_weights=[1, 0],optimizer=opt)


history = model.fit(x=S_train, y=[y_train, mel_train], batch_size=2*batch_size, #Borrar el 10!
                            epochs=epochs, verbose=fit_verbose,
                            validation_split=0.0,
                            shuffle=True,
                            #callbacks=[MyCallback(alpha, beta),metrics_callback,save_fig])
                            callbacks=[metrics_callback,csv_logger])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 44, 513)      0                                            
__________________________________________________________________________________________________
dot_2 (DOT)                     (None, 44, 128)      65664       input_4[0][0]                    
__________________________________________________________________________________________________
rnn_2 (RNN)                     (None, 44, 128)      128         dot_2[0][0]                      
__________________________________________________________________________________________________
pcen_2 (PCEN)                   (None, 44, 128)      384         dot_2[0][0]                      
                                                                 rnn_2[0][0]                      
Total para

60000/60000 [==============================] - 34s 570us/step - loss: 0.2341 - model_5_loss: 0.2341 - model_4_loss: 2.4743e-04
F1 = 0.4439, ER = 0.6730 -  Best val F1s: 0.4439 (IMPROVEMENT, saving)

Epoch 18/101
60000/60000 [==============================] - 34s 570us/step - loss: 0.2323 - model_5_loss: 0.2323 - model_4_loss: 2.5592e-04
F1 = 0.4206, ER = 0.6968 - Best val F1s: 0.4439 (16)

Epoch 19/101
60000/60000 [==============================] - 34s 571us/step - loss: 0.2304 - model_5_loss: 0.2304 - model_4_loss: 2.6301e-04
F1 = 0.4396, ER = 0.6783 - Best val F1s: 0.4439 (16)

Epoch 20/101
60000/60000 [==============================] - 34s 570us/step - loss: 0.2274 - model_5_loss: 0.2274 - model_4_loss: 2.7051e-04
F1 = 0.4426, ER = 0.6751 - Best val F1s: 0.4439 (16)

Epoch 21/101
60000/60000 [==============================] - 34s 571us/step - loss: 0.2259 - model_5_loss: 0.2259 - model_4_loss: 2.8027e-04
F1 = 0.4475, ER = 0.6681 -  Best val F1s: 0.4475 (IMPROVEMENT, saving)

Epoch 2

60000/60000 [==============================] - 34s 570us/step - loss: 0.1725 - model_5_loss: 0.1725 - model_4_loss: 4.7879e-04
F1 = 0.4730, ER = 0.6256 - Best val F1s: 0.4746 (41)

Epoch 59/101
60000/60000 [==============================] - 34s 571us/step - loss: 0.1708 - model_5_loss: 0.1708 - model_4_loss: 4.8101e-04
F1 = 0.4742, ER = 0.6210 - Best val F1s: 0.4746 (41)

Epoch 60/101
60000/60000 [==============================] - 34s 569us/step - loss: 0.1708 - model_5_loss: 0.1708 - model_4_loss: 4.8613e-04
F1 = 0.4756, ER = 0.6175 -  Best val F1s: 0.4756 (IMPROVEMENT, saving)

Epoch 61/101
60000/60000 [==============================] - 34s 569us/step - loss: 0.1690 - model_5_loss: 0.1690 - model_4_loss: 4.8764e-04
F1 = 0.4744, ER = 0.6203 - Best val F1s: 0.4756 (59)

Epoch 62/101
60000/60000 [==============================] - 34s 571us/step - loss: 0.1693 - model_5_loss: 0.1693 - model_4_loss: 4.9245e-04
F1 = 0.4679, ER = 0.6232 - Best val F1s: 0.4756 (59)

Epoch 63/101
60000/60000 

60000/60000 [==============================] - 34s 572us/step - loss: 0.1369 - model_5_loss: 0.1369 - model_4_loss: 6.2026e-04
F1 = 0.4816, ER = 0.5913 - Best val F1s: 0.4868 (91)

Epoch 101/101
60000/60000 [==============================] - 34s 569us/step - loss: 0.1362 - model_5_loss: 0.1362 - model_4_loss: 6.2343e-04
F1 = 0.4845, ER = 0.5949 - Best val F1s: 0.4868 (91)

